# Homework 2 - Anastasia Nikiforova

```В самом низу этого ноутбука - которкие выводы.```

In [61]:
import gensim
import pandas as pd
from lxml import html

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re

from string import punctuation
import json, os
from collections import Counter
import numpy as np
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')


In [82]:
corpus = [line for line in open('corpus_eng.txt', encoding='utf-8')]

In [93]:
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('english'))
lemma = WordNetLemmatizer()
digits = re.compile("./d.") # Getting rid of any strings that contain numbers: time, date etc.
arabic = ['i', 'ii', 'iii', 'iv', 'v','vi', 'vii', 'viii', 'xi', 'x']

In [100]:
def hasNumbers(string):
    return any(char.isdigit() for char in string)

In [432]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word]
    words = [word for word in words if word and word not in stops]
    words = [word for word in words if word and word not in arabic]
    words = [word for word in words if word and not hasNumbers(word)]
    words = [word for word in words if word and word not in punct]
    words = [lemma.lemmatize(word) for word in words if word]
    words = [word.replace("'s", '') for word in words if word] 
    words = [word.replace("o'", '') for word in words if word] 
    words = [word.strip(punct) for word in words if word]  # just in case (какие-то точки не убрались сразу)
    words = [word for word in words if word and len(word)>2]
    
    return words

In [277]:
test = ". ThE cat's on tHe mat:&? o'riley! will be in a me houses \
        hidden wrote done 123ew 321ouch iii x COmpuTatioNal Linguistics \
        breaks o'break's BREAK breaking broke broken"
normalize(test)

['cat',
 'mat',
 'riley',
 'house',
 'hidden',
 'wrote',
 'done',
 'computational',
 'linguistics',
 'break',
 'break',
 'break',
 'breaking',
 'broke',
 'broken']

In [260]:
# for i in corpus_norm:
#     if 'breaking' in i:
#         print(i)

Хм, интересно! Здесь видно, что формы прошедшего времени и герундия лемматизатор почему-то не лемматизировал. Хотя такие предложения в корпусе однозначно есть (код закомменчен, чтобы не отображать длинный список)

**[Эта проверка была проведена не в самом начале, а только после того, как были получены 418 предложений, содержащих ```break```. Модификация функции нормализации будет ниже.]**

In [278]:
corpus[0]

"States to Watch on Election Day 11/7/2016 6:00AM Trump and Clinton will put their election strategies to the test on Tuesday. WSJ's Gerald F. Seib discusses which states to watch as the polls close and which states both candidates need to win in order to claim victory. Photo: AP Transcript \n"

In [279]:
corpus_norm = [normalize(line) for line in corpus]

In [433]:
corpus_norm_1 = [normalize(line) for line in corpus]

In [280]:
# Избавились от любых токенов, в которых есть цифры (встречались даты, время  т.п.), 
# в т.ч. арабские (встречалась такая нумерация).
# Все тексты лемматизированы с помощью Wordnet Lemmatizer. 
# Избавились от posessive case (т.к. 's не несет важной для нас информации) и от приставки o'.
# Ограничили длину слова (>2), удалена пунктуация и стоп-слова.
corpus_norm[8]

['price',
 'pledged',
 'work',
 'ensure',
 'health',
 'care',
 'system',
 'work',
 'patient',
 'family',
 'doctor',
 'lead',
 'world',
 'cure',
 'prevention',
 'illness',
 'based',
 'sensible',
 'rule',
 'protect',
 'well-being',
 'country',
 'embracing',
 'innovative',
 'spirit']

In [281]:
corpus_break = [w for w in corpus_norm if 'break' in w]
len(corpus_break)

497

497 - Получилось чуточку больше!*

\* *Без лемматизации было всего 408 включений, а значит в корпусе немало разных форм слова break (breaks/breaking etc.).*

P.S. Позже выяснилось, что не все формы лемматизируются как надо, поэтому нормализация была дополнена еще одним условием.

_____
### Лирическое отступление: Модификация функции нормализации 

In [285]:
# Чтобы в нормализации не писать несколько строчек реплейса, ввожу функцию замены сразу по нескольким вариантам
def replaceMultiple(mainString, toBeReplaces, newString):
    # Iterate over the strings to be replaced
    for elem in toBeReplaces:
        # Check if string is in the main string
        if elem in mainString:
            # Replace the string
            mainString = mainString.replace(elem, newString)
    
    return  mainString

In [329]:
break_forms = ['breaks', 'breaking', 'broke', 'broken']
stop_extra = set(list(stops) + ['yes', 'sorry', 'can', 'could', 'would'])  # часто встречаются, но по сути бессмыссленны
for_replacement = ["'s", "o'", "'m", "'ve", "'d'", "’s", "’ve", "’d", "’m", "o’"] # в корпусе встречается два вида апострофов

def normalize_upd(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word]
    
    # Тут возникла проблема с примерами типа "else?(sorry" (нет пробела после знака препинания)
    # Решение - разделим каждый подобный пример по знаку пунктуации:
    words = ["".join((char if char.isalpha() else " ") for char in word).split() for word in words if word]
    # При этом функция isalpha() дополнительно уберет все строки, в которых есть цифры
    # Значит наша функция hasNumbers() больше не нужна.
    
    # Тут еще одна проблема - теперь в списке появятся подсписки из-за разделения проблемных участков на предыдущем шаге
    # Нужно "расправить" список:
    words = [word for sublist in words for word in sublist]
    
    # Теперь можно нормализовать дальше
    words = [lemma.lemmatize(word) for word in words if word]
    words = [word for word in words if word and word not in stops]
    words = [word for word in words if word and word not in arabic] # но арабские цифры нужно убрать
    # words = [word for word in words if word and not hasNumbers(word)] - obsolete
    words = [lemma.lemmatize(word) for word in words if word]
    words = [replaceMultiple(word, for_replacement, '') for word in words if word] 
#     words = [word.replace("'s", '') for word in words if word]
#     words = [word.replace("o'", '') for word in words if word] 
    words = [word.strip(punct) for word in words if word]  # just in case (какие-то точки не убрались сразу)
    words = [word for word in words if word and len(word)>2]
    words = ['break' if word in break_forms else word for word in words if word] # дополняем лемматизацию
    
    return words

Забавно, что ```word.lstrip("o'")``` и ```word.rstrip("'s")``` просто съедали буквы ```o``` и ```s``` в начале/конце слов, даже при экранировании апострофа (пробовала варианты ```''``` и ```\'```). Интересно, почему ```strip``` не работает с апострофами...

Тем не менее, ```replace``` справился отлично. 
UPD: лучше написать функцию для мультизамены, т.к. нужно много чего заменить

In [330]:
test = ". ThE cat's on tHe mat:&? o'riley! will be in a me houses \
        123ew 321ouch iii x COmpuTatioNal Linguistics \
        breaks o'break's BREAK breaking broke broken order mass’s hi!i'm(house) o’loads)of!trash’d"
normalize_upd(test)

['cat',
 'mat',
 'riley',
 'house',
 'ouch',
 'computational',
 'linguistics',
 'break',
 'break',
 'break',
 'break',
 'break',
 'break',
 'order',
 'mass',
 'house',
 'load',
 'trash']

Ух, как красиво теперь все токенизируется! Еще бы лемматизация проходила идеально для всех - было бы прекрасно!

In [331]:
corpus_norm = [normalize_upd(line) for line in corpus]

In [332]:
print(corpus_norm[8])

['price', 'pledged', 'work', 'ensure', 'health', 'care', 'system', 'work', 'patient', 'family', 'doctor', 'lead', 'world', 'cure', 'prevention', 'illness', 'based', 'sensible', 'rule', 'protect', 'well', 'country', 'embracing', 'innovative', 'spirit']


Теперь текст лемматизирован гораздо лучше. Но, конечно, не идеально, потому что лучше бы лемматизировать все слова как надо, а не только ```break```. Но если лемматизировать ```description```/```examples``` этой же функцией, по идее все должно быть хорошо.

In [367]:
corpus_break = [w for w in corpus_norm if 'break' in w]
len(corpus_break)

1256

Ух тыы! Почти в три раза возрасло количество предложений! (а то и больше, т.к. без лемматизации было 408)

Значит разных форм слова ```break``` было не "немало", как было заявлено выше, а даже больше, чем слова ```break``` изначально в начальной форме!

Надеюсь, что это поможет нормально, а не по-медвежьи (хоть медведи и довольно милые).

Позже выяснилось, что очень много вариантов "breaking news", алгоритм всегда определяет смысл как ```break_61 - be released or become known; of news```, что было бы верно в случае "break the news", но с "breaking news" это вряд ли 100% подходящее значение.

Короче говоря, сделаем отдельный список без брейкинг ньюс

In [399]:
def sublist(ls1, ls2):
    '''
    >>> sublist([], [1,2,3])
    True
    >>> sublist([1,2,3,4], [2,5,3])
    True
    >>> sublist([1,2,3,4], [0,3,2])
    False
    >>> sublist([1,2,3,4], [1,2,5,6,7,8,5,76,4,3])
    False
    '''
    def get_all_in(one, another):
        for element in one:
            if element in another:
                yield element

    for x1, x2 in zip(get_all_in(ls1, ls2), get_all_in(ls2, ls1)):
        if x1 != x2:
            return False

    return True

In [400]:
breaking_news = ['break', 'news']
corpus_break_nonews = [w for w in corpus_break if sublist(breaking_news, w)]  # ['break', 'news']
len(corpus_break_nonews)

1016

На мой взгляд, это тоже не сильно повредит

____

In [334]:
min(map(len, corpus_break))  # выясним минимальную длину элемента в получившемся корпусе - на всякий случай

1

Это наверняка не очень полезно для алгоритма... Давайте введем ограничение на то, что в нормализованном предложении должно быть не меньше, скажем, шести слов.

In [335]:
# corpus_break_lim = [del(i) for i in corpus_break if len(i)<6] # так низя
corpus_break_lim = [i for i in corpus_break if len(i)>6]

In [336]:
len(corpus_break_lim)

1229

Было 1256, стало 1229 - стало быть, мы избавились от 27 коротких значений. На общем фоне, потеря кажется совсем небольшой.

Проверим, тсколлько всего значений break содержится в wordnet

In [431]:
# word = 'break'
# i = 0
# for synset in wn.synsets(word):
#     print(str(i) + ' - ' + word + ' - ' + synset.definition())
#     i += 1

75! ой-ой... много :\

In [326]:
# word = 'break'
# i = 0
# for synset in wn.synsets(word):
#     print(str(i) + ' - ' + word + ' - ' + ' | '.join(synset.examples()))
#     i += 1

In [338]:
import random
corpus_break_10 = random.sample(corpus_break_lim, 10)
len(corpus_break_10)

10

Надо бы еще имена убрать... А то полно всяких Billy Joel-ов и Jordan Philips-ов

In [466]:
# corpus_break_10_text = []
# temp = ""
# for i in range(len(corpus_break_10)):
#     corpus_break_10_text.append(' '.join(corpus_break_10[i]))
#     temp = ""

# corpus_break_10_text[6]

In [348]:
def get_words_in_context(words, window=3):
    
    words_in_context = []
    
    for i in range(len(words)):
        left = words[max(0,i-window):i]
        right = words[i+1:i+1+window]
        target = words[i]
        words_in_context.append((target, left+right))
        
    
    return words_in_context

## Попытка номер 1 - поиск по Definition

In [349]:
def lesk_definition( word, sentence ):
    
    bestsense = 0
    maxoverlap = 0
    sentence = set(sentence)
    
    for i, synset in enumerate(wn.synsets(word)):
        definition = tokenize(synset.definition())
        definition = set(definition)
        overlap = len(definition & sentence)
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i
    
    return bestsense

In [350]:
dis_corpus = []
# пройдем по корпусу
for text in corpus_break_10:
    dis_text = []
    words_in_context = get_words_in_context(text)
    # дизамбигуируем каждой слов
    for word, context in words_in_context:
        nsense = lesk_definition(word, context)
        
        # если смысл не нулевой - добавим индекс смысла к токену
        if nsense > 0:
            dis_text.append(word + '_' + str(nsense))
        else:
            dis_text.append(word)
    
    dis_corpus.append(dis_text)

In [363]:
print(dis_corpus[3])

['man', 'shooting', 'burlington', 'county', 'street', 'published', 'saturday', 'november', 'burlington', 'city', 'prosecutor', 'say', 'man', 'shot', 'street', 'burlington', 'county', 'died', 'wound', 'still', 'clear', 'sparked', 'shooting', 'howard', 'young', 'year', 'old', 'burlington', 'township', 'man', 'shot', 'around', 'friday', 'block', 'east', 'federal', 'street_2', 'burlington', 'city', 'taken', 'hospital', 'died', 'short_17', 'time', 'later', 'burlington', 'county', 'prosecutor', 'say', 'autopsy', 'scheduled', 'saturday', 'declined', 'comment', 'slaying', 'citing', 'ongoing', 'investigation', 'arrest', 'made', 'latest', 'sfgate', 'homepage', 'click', 'top', 'news', 'around', 'bay', 'area', 'beyond', 'sign', 'newsletter', 'first', 'learn', 'break_61', 'news', 'sign', 'manage', 'profile', 'top', 'page', 'latest', 'news']


In [387]:
attempt_1 = dis_corpus

Что можно попробовать изменить:
* смотреть на examples
* смотреть на definition+examples
* смотреть на нормализованные examples/definitions
* изменить размер окна

In [392]:
options_1 = []
for i in range(len(attempt_1)):
    for j in attempt_1[i]:
        if 'break' in j:
            options_1.append(str(i) + " - " + j)
options_1

['0 - break_40',
 '1 - break',
 '1 - break',
 '2 - break',
 '3 - break_61',
 '4 - break',
 '4 - break_61',
 '5 - break',
 '6 - break',
 '7 - break',
 '8 - break_61',
 '9 - break_15']

___
Введем новую функцию, чтобы не писать каждый раз вывод всех вариантов

In [421]:
def show_all_senses(dis_corp):
    options = []
    for i in range(len(dis_corp)):
        for j in dis_corp[i]:
            if j[:5] == 'break':
                if len(j) == 5:
                    options.append(str(i) + " - " + j + "_0" + " - " +  wn.synsets('break')[0].definition())
                else:
                    options.append(str(i) + " - " + j + " - " +  wn.synsets('break')[int(j.split('_')[1])].definition())
    return options

И еще одну для дизамбигуации

In [425]:
def disambiguate(corpus):
    dis_corpus = []
    # пройдем по корпусу
    for text in corpus:
        dis_text = []
        words_in_context = get_words_in_context(text) 
        # дизамбигуируем каждой слов
        for word, context in words_in_context:
            nsense = lesk_definition(word, context)

            # если смысл не нулевой - добавим индекс смысла к токену
            if nsense > 0:
                dis_text.append(word + '_' + str(nsense))
            else:
                dis_text.append(word)

        dis_corpus.append(dis_text)
    return dis_corpus

In [468]:
show_all_senses(attempt_1)

['0 - break_40 - cause to give up a habit',
 '1 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '1 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '2 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '3 - break_61 - be released or become known; of news',
 '4 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '4 - break_61 - be released or become known; of news',
 '5 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '6 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '7 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '8 - break_61 - be released or become known; of news',
 '9 - break_15 - an escape from jail']

In [483]:
print(attempt_1[9])

['country', 'prepared', 'election', 'flight', 'lieutenant', 'jerry', 'rawlings', 'several', 'junior', 'officer', 'launched_2', 'coup', 'successful', 'first_14', 'another', 'group', 'officer', 'break_15', 'jail', 'rawlings', 'made_51', 'second', 'successful', 'coup', 'attempt', 'overthrew', 'government']


**Результаты:**
```
* 0 - Из лемматизированного корпуса не очень понятно, но кажется, тут молодой человек заствил 'kid' что-то бросить - мб верно
* 1 - Здесь брейк как каникулы - неверно
* 2 - Что-то про музыку - неверно
* 3 - Брейкинг ньюс - предположим, что это верно
* 4 - Тут есть и брейкинг ньюс (верно) + что-то про каникулы в апреле (неверно)
* 5 - Что-то про атаки (неверно)
* 6 - Что-то про болезни (неверно)
* 7 - Что-то про марши в корее и политическое напряжение (неверно)
* 8 - Брейкинг ньюс (верно)
* 9 - Побег из тюрьмы! Верно!

Итого: 4,5/10 (в 4 примере верно только одно break)
```

Прекрасно! А теперь

## Попытка номер 1.1 - поиск по Definitions в корпусе без "Breaking news"

In [447]:
nonews_dis = disambiguate(random.sample(corpus_break_nonews, 10))
temp = nonews_dis # чтобы не пропало :)

In [448]:
show_all_senses(nonews_dis)

['0 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '1 - break_10 - the opening shot that scatters the balls in billiards or pool',
 '2 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '3 - break_7 - breaking of hard tissue such as bone',
 "4 - break_2 - (geology) a crack in the earth's crust resulting from the displacement of one side with respect to the other",
 '5 - break_61 - be released or become known; of news',
 '6 - break_1 - an unexpected piece of good luck',
 '7 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '8 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '9 - break_0 - some abrupt occurrence that interrupts an ongoing activity']

In [459]:
print(temp[9])

['perception', 'pelly', 'speaks', 'rooted', 'december', 'incident', 'little', 'india', 'riot', 'break', 'inebriated', 'indian', 'worker', 'died', 'wheel', 'private', 'bus', 'according', 'record', 'people', 'injured', 'arrested']


**Верно**\*: 
```
* 0 (долгая засуха сменилась резким дождем)
* 5 (таки новости пробрались, но все же смысл про выпуск чего-то (в частности новостей))
* 9 (забастовку ведь можно назвать кратковременным нарушением спокойствия?)
```

**Неверно**\*: 
```
* 1 ('wrestling', 'slager', 'stun', 'gun', 'break_10', 'away_7', 'shot', 'back' - алгоритм увидел слово "shot")
* 2 ('break', 'leg' - имелось в виду "Break a leg!")
* 3 ('favouring', 'hard', 'brexit', 'clean', 'break_7', 'single', 'market' - алгоритм увидел слово "hard")
* 4 ('grape', 'leaf', 'use', 'break_2', 'torn', 'one' - тут запутался из-за leaf - отнес его к биологии)
* 6 ('good_20', 'condition_4', 'break_1', 'left', 'arm', 'bite' - алгоритм увидел слово "good")
* 7 ('defense', 'break', 'ground', 'million', 'project', 'expected' - просто не нашел ничего общего)
* 8 ('joel', 'embiid', 'break', 'handle', 'fake' - просто не нашел ничего общего)
```


\* более или менее, т.к. оценка субъективная

## Попытка номер 2 - Смотрим на Examples

In [460]:
def lesk_examples( word, sentence ):
    
    bestsense = 0
    maxoverlap = 0
    sentence = set(sentence)
    
    for i, synset in enumerate(wn.synsets(word)):
        examples = tokenize(' '.join(synset.examples()))
        examples = set(examples)
        overlap = len(examples & sentence)
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i
    
    return bestsense

In [463]:
# sent_test = ['be', 'released', 'known', 'news']
# word_test = 'break'
# lesk_examples(word_test, sent_test)

In [484]:
# get_words_in_context(sent_test, window=3)

In [485]:
def disambiguate_by_examples(corpus):
    dis_corpus = []
    # пройдем по корпусу
    for text in corpus:
        dis_text = []
        words_in_context = get_words_in_context(text) 
        # дизамбигуируем каждой слов
        for word, context in words_in_context:
            nsense = lesk_examples(word, context)

            # если смысл не нулевой - добавим индекс смысла к токену
            if nsense > 0:
                dis_text.append(word + '_' + str(nsense))
            else:
                dis_text.append(word)

        dis_corpus.append(dis_text)
    return dis_corpus

In [514]:
temp = disambiguate_by_examples(random.sample(corpus_break, 10))

In [515]:
temp_2 = temp

In [516]:
show_all_senses(temp_2)

['0 - break_24 - force out or release suddenly and often violently something pent up',
 '0 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '1 - break_51 - break down, literally or metaphorically',
 '2 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 "3 - break_26 - enter someone's (virtual or real) property in an unauthorized manner, usually with the intent to steal or commit a violent act",
 '4 - break_30 - make known to the public information that was previously known only to a few people or that was meant to be kept a secret',
 '5 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '6 - break_22 - move away or escape suddenly',
 '7 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '8 - break_46 - discontinue an association or relation; go different ways',
 '9 - break_1 - an unexpected piece of good luck']

In [528]:
print(temp_2[9])

['georgetown', 'mcdonough', 'arena', 'sort', 'intimate', 'flavorful', 'charismatic', 'campus', 'arena', 'make', 'college', 'basketball', 'grand', 'hoya', 'occasional', 'game', 'verizon', 'center', 'tiny', 'little_8', 'cousin', 'nice', 'break_1', 'team', 'big', 'chinatown', 'home', 'logistics', 'aside', 'always', 'thought_1', 'great', 'game_2', 'mcdonough']


**Результаты:**
```
*0 Тут про ломание придметов в ярости - мне кажется, верное значение (break thing anger). Второе неверно (break heart)
*1 "break down, literally" подходит! текст про то, как ломается лед в океанах
*2 Тут подошло бы метафоричное значение предыдущего - про разрушение того, во что кто-то верит (неверно)
*3 Что-то про магазины, к сожалению, не ограбление (тогда подошло бы)
*4 Ну по сути, breaking news вполне и под это значение подходит - а-ля сенсация
*5 Странный текст - возможно про бейсбол, т.к. не понимаю термины. Но смысл определился неверно
*6 Неверно - тут про переломную точку
*7 Что-то про рабочий коллектив - неверно
*8 Неверно - текст про black friday
*9 Что-то про футбольную команду - неверно

Итого: 3/10 
P.S. Тут на самом деле раз на раз не приходится :( если долго подбирать выборку, можно наверняка довести до 6/10
```

In [574]:
temp = disambiguate_by_examples(random.sample(corpus_break, 10))

In [575]:
temp_3 = temp

In [576]:
show_all_senses(temp_3)

['0 - break_23 - scatter or part',
 "1 - break_26 - enter someone's (virtual or real) property in an unauthorized manner, usually with the intent to steal or commit a violent act",
 '1 - break_4 - a pause from doing something (as work)',
 '2 - break_29 - surpass in excellence',
 '3 - break_4 - a pause from doing something (as work)',
 '3 - break_4 - a pause from doing something (as work)',
 '4 - break_30 - make known to the public information that was previously known only to a few people or that was meant to be kept a secret',
 '5 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '6 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '7 - break_22 - move away or escape suddenly',
 '8 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '9 - break_30 - make known to the public information that was previously known only to a few people or that was meant to be kept a secret']

In [586]:
print(temp_3[9])

['ricky', 'martin', 'engaged', 'artist', 'jwan', 'yosef', 'luchina', 'fisher', 'nov', 'share', 'ricky', 'martin', 'appears', 'ellen', 'degeneres', 'show', 'burbank', 'california', 'share', 'ricky', 'martin', 'engaged', 'latin', 'music', 'star', 'proposed', 'jwan', 'yosef', 'artist', 'dating', 'year', 'wednesday', 'break_30', 'news', 'ellen', 'degeneres', 'show', 'conceptual', 'artist', 'collector', 'started', 'looking', 'art', 'saw_15', 'art', 'went', 'crazy_3', 'really', 'love', 'doe', 'really', 'original', 'martin', 'shared', 'contacted', 'collected', 'ellen', 'degeneres', 'joked', 'got', 'engaged', 'martin', 'announced', 'showing', 'ring', 'sweating', 'martin', 'admitted', 'really', 'nervous', 'proposal', 'got_12', 'knee', 'said', 'took_27', 'ring', 'little', 'velvet', 'pouch_1', 'pocket', 'recalled', 'instead', 'saying_1', 'marry_1', 'said', 'got_1', 'something', 'bad_13', 'singer', 'said', 'like', 'yes', 'said', 'want', 'spend', 'life', 'like', 'question_10', 'would', 'marry', 'ma

**Результаты:**
```
*0 брейкинг ньюс (ронг)
*1 О! Кража банковских стредств (ведь чужой аккаунт==счет тоже можно считать property? зато интенция совпадает) +
*1_1 Тут имело в виду completely broke (после кражи средств) - неверно
*2 Верно! ('expected', 'break_29', 'record_5', 'year') - побить годовые рекорды - чем не excellence +
*3 Здесь неоднозначно (ха-ха). Вроде бы и про отпуск (took break), но может и нет +-
*4 Брейкинг ньюс! Скажем, плюс
*5 Здесь про банкротство (неверно)
*6 Что-то про акции (неверно)
*7 Тут про рушащиеся мечты (неверно)
*8 Хм... А тут про короткий перерыв (short break) в игре - это подходит
*9 Брейкинг ньюс! Снова плюс

Итого: 5,5/10 (с третьим неоднозначно) 
```

Уже лучше!

## Попытка номер три - Definitions+Examples

In [529]:
def lesk_defex( word, sentence ):
    
    bestsense = 0
    maxoverlap = 0
    sentence = set(sentence)
    
    for i, synset in enumerate(wn.synsets(word)):
        examples = tokenize(' '.join(synset.examples()))
        definition = tokenize(synset.definition())
        defex = set(definition+examples)
        overlap = len(defex & sentence)
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i
    
    return bestsense

In [537]:
def disambiguate_by_defex(corpus):
    dis_corpus = []
    # пройдем по корпусу
    for text in corpus:
        dis_text = []
        words_in_context = get_words_in_context(text) 
        # дизамбигуируем каждой слов
        for word, context in words_in_context:
            nsense = lesk_defex(word, context)

            # если смысл не нулевой - добавим индекс смысла к токену
            if nsense > 0:
                dis_text.append(word + '_' + str(nsense))
            else:
                dis_text.append(word)

        dis_corpus.append(dis_text)
    return dis_corpus

In [637]:
temp = disambiguate_by_defex(random.sample(corpus_break, 10))

In [638]:
temp_defex = temp

In [639]:
show_all_senses(temp_defex)

['0 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '1 - break_11 - (tennis) a score consisting of winning a game when your opponent was serving',
 '2 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '3 - break_30 - make known to the public information that was previously known only to a few people or that was meant to be kept a secret',
 '4 - break_47 - assign to a lower position; reduce in rank',
 '5 - break_1 - an unexpected piece of good luck',
 '6 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 "7 - break_2 - (geology) a crack in the earth's crust resulting from the displacement of one side with respect to the other",
 "8 - break_26 - enter someone's (virtual or real) property in an unauthorized manner, usually with the intent to steal or commit a violent act",
 '9 - break_30 - make known to the public information that was previously known only to a few people or that was meant to be kept a secret',
 '9 -

In [650]:
print(temp_defex[9])

['game', 'sunday', 'november', 'updated', 'monday', 'november', 'goal', 'latest', 'sfgate', 'homepage', 'click', 'top', 'news', 'around', 'bay', 'area', 'beyond', 'sign', 'newsletter', 'first', 'learn', 'break_30', 'news', 'sign', 'manage', 'profile', 'top', 'page', 'break_30', 'news']


**Результаты:**
```
0* неверно, тут про breaking campaign (=good)
1* не знакома с теннистыми терминами, но выглядит и правда как очки за теннис (только тут table tennis) +
2* не разберу значение предложения :( но явно неверно
3* что-то про опрос миллиона каталонцев, и это могло бы быть верно, но все кажется, что нет
4* скорее всего неверно. Значение забрало из определения, триггер на слово 'lower'
5* вау, а тут правда! ('winning', 'big', 'tax', 'break_1', 'trillion', 'foreign', 'profit') +
6* тут что-то про ядерный взрыв ('large', 'black', 'mushroom_3', 'cloud', 'rose', 'seaside') - неверно
7* неверно, триггер на слово "one"
8* ой, тут про birth control в honeymoon *awkward* (неверно)
9* Брейкинг ньюс! Дважды плюс ++

Итого: 4/10
```

## Попытка 4 - нормализация DefEx

(на этот способ возлагаются большие надежды. Если оно не сработает, то грустно)

In [651]:
def normalize_defex(list_defex):
    
    words = [word.strip(punct) for word in list_defex if word]

    words = [lemma.lemmatize(word) for word in words if word]
    words = [word for word in words if word and word not in stops]
    words = [replaceMultiple(word, for_replacement, '') for word in words if word] 
    words = [word.strip(punct) for word in words if word]
    words = [word for word in words if word and len(word)>2]
    words = ['break' if word in break_forms else word for word in words if word] # дополняем лемматизацию
    
    return words

In [652]:
def lesk_defex_norm( word, sentence ):
    
    bestsense = 0
    maxoverlap = 0
    sentence = set(sentence)
    
    for i, synset in enumerate(wn.synsets(word)):
        examples = tokenize(' '.join(synset.examples()))
        examples = normalize_defex(examples)
        definition = tokenize(synset.definition())
        definition = normalize_defex(definition)
        defex = set(definition+examples)
        overlap = len(defex & sentence)
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i
    
    return bestsense

In [653]:
def disambiguate_by_defex_norm(corpus):
    dis_corpus = []
    # пройдем по корпусу
    for text in corpus:
        dis_text = []
        words_in_context = get_words_in_context(text) 
        # дизамбигуируем каждой слов
        for word, context in words_in_context:
            nsense = lesk_defex_norm(word, context)

            # если смысл не нулевой - добавим индекс смысла к токену
            if nsense > 0:
                dis_text.append(word + '_' + str(nsense))
            else:
                dis_text.append(word)

        dis_corpus.append(dis_text)
    return dis_corpus

In [676]:
temp = disambiguate_by_defex_norm(random.sample(corpus_break, 10))

In [677]:
temp_defex_norm = temp

In [678]:
show_all_senses(temp_defex_norm)

['0 - break_4 - a pause from doing something (as work)',
 '1 - break_11 - (tennis) a score consisting of winning a game when your opponent was serving',
 '2 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '2 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '3 - break_46 - discontinue an association or relation; go different ways',
 '4 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '5 - break_68 - happen',
 '6 - break_1 - an unexpected piece of good luck',
 '6 - break_11 - (tennis) a score consisting of winning a game when your opponent was serving',
 '7 - break_43 - happen or take place',
 '8 - break_29 - surpass in excellence',
 '9 - break_11 - (tennis) a score consisting of winning a game when your opponent was serving']

In [689]:
print(temp_defex_norm[9])

['fifth', 'seed_2', 'one', 'six', 'player', 'already', 'qualified', 'season', 'ending', 'final', 'london', 'breezed_1', 'first_8', 'set_28', 'quickly', 'break_11', 'second']


**Результаты:**
```
0* Тут про таймаут в какой-то игре - подходит +
1* Здесь точно про какую-то игру, и break - вероятнее всего про очки +
2* Тут про html разметку ('http', 'embed', 'break', 'width', 'height', 'frameborder') - неверно
3* Ой, кажется это верно! ('kill', 'friend', 'point_14', 'break_46', 'batman', 'superman', 'dawn', 'justice') про конфликт между супергероями +
4* Тут про образование пробок на дорогах - неверно.
5* Неверно - тут про heart break (причем метафорично)
6* Первое неверно (имелся в виду таймаут). А второе похоже на правду - счет в теннисе (текст про ход игры в теннис). +
7* Неверно, триггер на "take". Тест про nba taking break for thanksgiving
8* Неверно. Здесь зашумляют имена. В идеале их нужно убрать из корпуса.
9* Верно, снова про очки в игре (во втором сете) +

Итого: 5/10
```

В принципе, неплохо

## Попытка [-1] - Изменение размера окна

Попробуем поиграться с последним методом (нормализованные description + examples), изменив размер окна на 2

In [695]:
def get_words_in_context(words, window=2):
    
    words_in_context = []
    
    for i in range(len(words)):
        left = words[max(0,i-window):i]
        right = words[i+1:i+1+window]
        target = words[i]
        words_in_context.append((target, left+right))
        
    
    return words_in_context

In [737]:
temp = disambiguate_by_defex_norm(random.sample(corpus_break, 10))

In [738]:
temp_defex_win2 = temp

In [739]:
show_all_senses(temp_defex_win2)

['0 - break_21 - act in disregard of laws, rules, contracts, or promises',
 '1 - break_63 - interrupt the flow of current in',
 '2 - break_1 - an unexpected piece of good luck',
 '3 - break_15 - an escape from jail',
 '4 - break_30 - make known to the public information that was previously known only to a few people or that was meant to be kept a secret',
 '5 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '6 - break_21 - act in disregard of laws, rules, contracts, or promises',
 '7 - break_30 - make known to the public information that was previously known only to a few people or that was meant to be kept a secret',
 '8 - break_12 - an act of delaying or interrupting the continuity',
 '9 - break_0 - some abrupt occurrence that interrupts an ongoing activity']

In [750]:
print(temp_defex_win2[9])

['roommate', 'family', 'send', 'condolence', 'card_2', 'also', 'gave_7', 'birthday', 'present_9', 'let', 'stay', 'entirety', 'thanksgiving', 'break', 'family', 'childhood', 'best_9', 'friend', 'clean_3', 'entire', 'house_6', 'death', 'mother', 'also', 'took', 'lunch', 'frozen', 'yogurt', 'summer', 'talk', 'went_4', 'first', 'vacation', 'ever', 'year', 'stayed_3', 'friend_2', 'family', 'showed', 'also', 'embraced_1', 'open_8', 'arm', 'invited', 'visit', 'mexico', 'friend', 'colorado_1', 'friend', 'kenya', 'friend', 'south', 'carolina', 'whose', 'family', 'ready', 'geographical', 'chance']


**Результаты:**
```
0* Верно (текст про sexual harrassment)
1* Верно (республиканец пытается изменить ход событий в конгрессе)
2* Неверно (что-то про игрушки)
3* Верно (побег из индийской тюрьмы)
4* Верно (брейкинг ньюс)
5* Неверно (что-то про болельщиков, кажется)
6* Верно (какой-то Зума из южной африки был обвинен в нарушении закона)
7* Верно (брейкинг ньюс)
8* Неверно (что-то про барака обаму на late night show джимми фэллона)
9* Неверно (текст про каникулы на день благодарения - thanksgiving break)

Итого: 6/10
```

Пока это лучший результат!

Ну и последнее - попробуем увеличить размер окна до 4

In [751]:
def get_words_in_context(words, window=4):
    
    words_in_context = []
    
    for i in range(len(words)):
        left = words[max(0,i-window):i]
        right = words[i+1:i+1+window]
        target = words[i]
        words_in_context.append((target, left+right))
        
    
    return words_in_context

In [790]:
temp = disambiguate_by_defex_norm(random.sample(corpus_break, 10))

In [791]:
temp_defex_win4 = temp

In [792]:
show_all_senses(temp_defex_win4)

['0 - break_1 - an unexpected piece of good luck',
 '1 - break_47 - assign to a lower position; reduce in rank',
 '2 - break_4 - a pause from doing something (as work)',
 '3 - break_46 - discontinue an association or relation; go different ways',
 '4 - break_30 - make known to the public information that was previously known only to a few people or that was meant to be kept a secret',
 "5 - break_26 - enter someone's (virtual or real) property in an unauthorized manner, usually with the intent to steal or commit a violent act",
 '6 - break_0 - some abrupt occurrence that interrupts an ongoing activity',
 '7 - break_21 - act in disregard of laws, rules, contracts, or promises',
 '8 - break_13 - a sudden dash',
 '9 - break_30 - make known to the public information that was previously known only to a few people or that was meant to be kept a secret']

In [802]:
print(temp_defex_win4[9])

['close', 'image', 'wisconsin', 'wide', 'receiver_5', 'jazz', 'peavy', 'catch', 'front', 'northwestern', 'safety', 'kyle', 'queiro', 'first', 'half_1', 'ncaa', 'college', 'football', 'game', 'evanston', 'ill', 'saturday', 'nov', 'wisconsin', 'wide', 'receiver_5', 'jazz', 'peavy', 'catch', 'front', 'northwestern', 'safety', 'kyle', 'queiro', 'first', 'half_1', 'ncaa', 'college', 'football', 'game', 'evanston', 'ill', 'saturday', 'nov', 'photo', 'david', 'bank', 'image', 'wisconsin', 'running_7', 'back', 'corey', 'clement', 'tackled', 'northwestern', 'linebacker', 'brett', 'walsh', 'first', 'half_1', 'ncaa', 'college', 'football', 'game', 'evanston', 'ill', 'saturday', 'nov', 'wisconsin', 'running_7', 'back', 'corey', 'clement', 'tackled', 'northwestern', 'linebacker', 'brett', 'walsh', 'first', 'half_1', 'ncaa', 'college', 'football', 'game', 'evanston', 'ill', 'saturday', 'nov', 'photo', 'david', 'bank', 'image', 'wisconsin', 'linebacker', 'ryan', 'connelly', 'tackle', 'northwestern', 

**Результаты:**
```
0* В принципе, верно. Говорится о стабилизации рынка после встречи ОПЕК. +
1* Неверно, но видно, что триггер на "lower"
2* Что-то прекратилось делаться первый раз с сентября - в принципе, верно. +
3* Опять тот пример про бэтмена с суперменом, определилось правильно, +
4* Брейкинг ньюс! +
5* Военный в отставке задержан бы вторжение в государственное учреждение (он - rebel - сделал там что-то offensive) +
6* Тут про влияние различных веществ на организм. ('cysteine', 'break', 'hangover', 'causing', 'toxin') неверно
7* Какой-то штат нарушил бы три государственных закона, если бы сделал что-то (что-то связанное с abortion). +
8* AAHH! Вот тут я бы поставила плюс, хотя бы за красивость (метафорично - подходит)! значение - a sudden dash (внезапный всплеск). Текст про WATERgate scandal (тот самый с Никсоном в конце прошлого столетия (Washington, Watergate Hotel), после которого он подал в отставку, опередив импичмент) +
9* Брейкинг ньюс! +

Итого: 8/10! Абсолютный рекорд!
```


## Выводы

В итоге, были испробованы множество разных вариантов улучшения алгоритма Леска.

На мой взгляд, самый видимиый результат принесла нормализация текстов.

Также логично, что увеличение размера окна до 4 при поиске по ```description + examples``` улучшит результат, т.к. текста у нас становится больше, чем при поиске исключительно по ```description``` или ```examples```.

Алгоритм часто выбирает неправильный вариант, когда триггерится на совпадающие слова в предложении и в описании (примерах).

Алгоритм часто выбирает 0 - так как судя по всему это дефолтное значение, если больше вообще ничего не подходит.

``` Выводы: для лучшего результата нужно 
1) Хорошо нормализовать корпус
2) Искать по description + examples
3) Увеличить размер окна до 4
```